# Xarray-simlab: create new processes

We'll see here how to create new xarray-simlab processes (Python classes) to extend model capabilities.

In [ ]:
import numpy as np
import xsimlab as xs
import xarray as xr


## Preparation

Import the `basic_model` from `fastscape`, and create a new xarray-simlab model instance by replacing the `flow` process (`SingleFlowRouter`) with the `MultipleFlowRouter` process, also available in `fastscape`.

Create a new simulation setup (note: for convenience you can use the `%create_setup` magic command), e.g, with the following values:

- grid size: 201 nodes along the x-axis, 101 nodes along y-axis
- grid length: 20 km along the x-axis, 10 km along the y-axis
- boundaries: periodic boundaries for the left/right grid borders, fixed value boundaries for the top/bottom borders
- vertical block uplift rate $U$: 0.001
- Stream-Power law's $K_f$ coefficient: 0.0001
- Linear diffusion $K_d$ coefficient: 0.1


Run the model with the settings created above (you can use xarray-simlab's [progress bar](https://xarray-simlab.readthedocs.io/en/latest/monitor.html#progress-bar) to track the simulation progress).

### Periodic forcing (time-varying erosion efficiency)

Create a time series for the Stream-Power law's $K_f$ coefficient, where its value oscillates at a given amplitude and frequency. The length and resolution of the time series should match the time steps used for the model run above. To create the time series, you can use numpy or xarray.

Plot the time series to ensure that it is ok (you can use the `xarray.Dataset.plot` extension, see the [plotting section](http://xarray.pydata.org/en/stable/plotting.html) in xarray's documentation).

Create a new simulation setup (reusing the previous one) with the $K_f$ coefficient time series, and run it.

### Domain-averaged erosion rates

Using the outputs from the last simulation, compute the time series of erosion rates averaged over the whole grid.

Plot the time series.

## Include pre/post-processing into the model

There are limitations with the time series that were computed above before and after running the simulation:

- What if we want to explore the effect of the amplitude and frequency on the results? We could create extra-dimensions for the $K_f$ coefficient array values (e.g., batch + time dimensions) and pass it a xarray-simlab new simulation setup, but it might become tedious and extra-dimensions may be confusing.
- If we want to have a high-resolution time series of domain-averaged erosion rates (i.e., at every time step), we would need to save the erosion rates for the whole grid at the same resolution, which might take a lot of memory for large grids and long simulations.

Fortunately, xarray-simlab models can be easily extended. We can create new processes that will compute those time series, and make it part of a model.

### Periodic forcing process

Create a new process class that implements sinusoidal variation of the $K_f$ coefficient at a given amplitude.

You'll need to declare:

- input variables for the average value of $K_f$, the amplitude and the frequency of the oscillation.
- one output (foreign) variable for the computed value of $K_f$ at each time step.

To compute the value of $K_f$, you'll also need to get the current time of the simulation. You can apply the [runtime](https://xarray-simlab.readthedocs.io/en/latest/_api_generated/xsimlab.runtime.html#xsimlab.runtime) decorator on the `run_step` function.

Test the new process created (just use the class without including it in a xarray-simlab model for now). You can for example create a time-series by calling `run_step` multiple times.

### Domain-averaged erosion rates process

Create a new process class that computes the domain averaged total erosion rate at each time step.

You'll need to declare:

- input (foreign) variables to get the total erosion rate values on grid nodes and the grid total area.
- one output value that computes the domain-averaged erosion rate.

Test the new process created (you can for example use a field of constant values of erosion rate).

### Update and run the model

Create a new model from the model used in the previous simulations, by including the two new processes created above.

Inspect the new model to see if the inputs have changed.

Visualize the graph of processes to see how the two new processes depend on other processes that were already in the model.

Create a new simulation setup. You can reuse one of the previous setups, with [xsimlab.update_vars](https://xarray-simlab.readthedocs.io/en/latest/_api_generated/xarray.Dataset.xsimlab.update_vars.html#xarray.Dataset.xsimlab.update_vars) and [xsimlab.filter_vars](https://xarray-simlab.readthedocs.io/en/latest/_api_generated/xarray.Dataset.xsimlab.filter_vars.html). Save model output snaptshots so that we can further get and plot the time series for both the $K_f$ coefficient and the domain-averaged erosion rate.

Plot the time series

## Bonus: sensitivity analysis

Explore the effect the amplitude and/or frenquency of the oscillations of $K_f$. See the [Run simulation batches](04_run_models.ipynb#run_simulation_batches) section of the notebook `04_run_models` for running multiple simulations with different values for model input values. You can visualize the outputs using `hvplot` (time series) or `ipyfastscape` (3D visualization).